### TR decomposition 

In [2]:
import numpy as np 
import copy

In [3]:
def svd_fix(x):
    n = x.shape[0]
    m = x.shape[1]
    
    if n > m:
        u, s, v = np.linalg.svd(x)
        
    else:
        u, s, v = np.linalg.svd(x.T)
        v, u = u, v
    
    return u, s, v

In [4]:
def truncation_index(vector, bound): 
    rnew = len(vector)
    norm_vector = np.cumsum(np.power(vector[::-1], 2))
    l = len(norm_vector)
    for i in range(l-1, 0, -1):
        if norm_vector[i] < bound**2:
            rnew = l - i
            break
    return rnew 

def divisors(n):
    d = []
    for i in range(1, int(n/2+1)):
        if n%i == 0: 
            d.append(i)
    d.append(n)
    return np.array(d)

In [5]:
def TRdecomp(A, prec, r0=0):
    """Converts a tensor A in full format into TR-format.
    Args:
    A: a tensor in full format 
    prec: a relative accuracy
    r0: the first TR-rank (must be a divisor of the first unfolding rank of A)
    Returns
    TR-format tensor representation 
    """

    s = np.shape(A)
    d = len(s)
    norm_error = prec / np.sqrt(d-1)
    cores = [[None] * 1  for _ in range(d)]
    C = copy.deepcopy(A)
    rold = 1
    
    n = s[0]
    C = np.reshape(C, (rold*n, np.size(C)//(rold*n)), order='F')
    U, S, V = svd_fix(C) 

    rnew = truncation_index(S, norm_error*np.linalg.norm(S))-1
    
    factors = divisors(rnew)
    if r0 == 0:
        if len(factors) > 2:
            r0 = int(np.random.choice(factors, 1))
        else:
            r0 = 1


    U = U[:,0:rnew]
    S = S[0:rnew]
    V = V[:, 0:rnew]
    
    Unew = np.zeros((rold*r0, n, rnew//r0))
    for index in range(1, r0+1):
        Unew[index - 1, :, :] = U[:, int((index-1)*(rnew//r0) + 1- 1):int(index*(rnew//r0))]
    
    cores[0] = Unew
    C =  np.diag(S) @ V.T

    Cnew = np.zeros((rnew//r0, np.size(C)//(rnew), r0))
    for index in range(1, r0+1):
        Cnew[:, :, index-1] = C[int((index-1)*(rnew//r0) + 1-1):int(index*(rnew/r0)), :]
    
    C = copy.deepcopy(Cnew)

    shapes = s[1:]
    C = C.reshape(rnew//r0, *shapes, r0, order='F')
    shapes = s[1:-1]
    C = C.reshape(rnew//r0, *shapes, int(s[-1]*r0), order='F')
    C = C.reshape(rnew//r0, int(np.size(C)//(rnew//r0)), order='F')
    rold = rnew//r0
    
    for k in range(1, d-1):
        n = s[k]
        C = C.reshape(rold*n, np.size(C)//(rold*n),  order='F')  
        U, S, V = svd_fix(C)
        rnew = truncation_index(S, norm_error*np.linalg.norm(S))-1
       
        U = U[:,0:rnew]
        S = S[0:rnew]
        V = V[:, 0:rnew]

        cores[k] = U.reshape(rold, n, rnew ,order='F')
        C = np.diag(S) @ V.T
        rold = rnew 
        
    C = C.reshape(rold, s[-1], r0, order='F')
    cores[d-1] = C
    
    return cores

In [7]:
recon = np.einsum("ijk,klm,mnp,pqi", *c)

### TR to full 

In [6]:
def TR_to_full(cores):
    """Converts a TR- representation into full format. 
    Args: 
    cores: TR-representation 
    Returns full-format tensor representation
    """
    a = cores[0] 
    d = len(cores) 
    r0, n, r1 = a.shape 
    ns = np.zeros(d)
    ns[0] = n 
    
    for k in range(1, d): 
        rold, n, rnew = a.shape 
        a = a.reshape(rold*n, rnew, order='F') 
        b = cores[k] 
        rnew, n, rnext = b.shape 
        ns[k] = n 
        b = b.reshape(rnew, int(np.size(b)/rnew), order='F')
        
        tmp = a @ b 
        a = tmp.reshape(rold, int(np.size(tmp)/(rold*rnext)), rnext, order='F') 
    a = a.reshape(r0, int(np.size(a)/(r0**2)), r0, order='F') 
        
    res = np.zeros(int(np. size(a)/r0**2)) 
        
    for k in range(r0): 
        res = res + a[k, :, k] 
        
    ns = list(map(int, ns)) 
    res = res.reshape(*ns, order='F') 
    return res 

### TR with shifts

In [7]:
def num_param_tr(cores): 
    '''Computes the number of parameters in the TR-representation
    '''
    d = len(cores)
    total_size = 0
    for i in range(d):
        s = cores[i].shape
        total_size = total_size + np.prod(s)
    return total_size

def ipermuteTR(cores, start_ind):
    '''Shifts the indices of a TR-representation
    start_ind-1 places to the right
    '''
    d = len(cores)
    res = [[None] * 1  for _ in range(d)]
    perm = np.concatenate((np.arange(start_ind-1, d), np.arange(0, start_ind-1)), axis=0)
    inv_perm = np.arange(len(perm))
    
    for i in range(len(perm)): 
        inv_perm[perm[i]] = i
        
    for k in range(d):
        res[k] = cores[inv_perm[k]]
    return res

In [8]:
def tr_shifts(A, prec, r0=0):
    '''Converts a tensor A in full format into TR-format,
    using relative accuracy prec.
    Loops over all cyclic shifts (with step size m)
    and divisors of the first unfolding rank of A
    and chooses the one with smallest storage cost.
    Args:
    A: a tensor in full format 
    prec: a relative accuracy
    r0: the first TR-rank (must be a divisor of the first unfolding rank of A)
    Returns
    TR-format tensor representation    
    '''
    
    s = np.shape(A)
    d = len(s)
    norm_error = prec / np.sqrt(d - 1)
    curr_small = float('Inf')
    cores = [[None] * 1  for _ in range(d)]
    curr_ind = -1
    m = 1
    
    for start_ind in range(1, d, m):
        perm = np.concatenate((np.arange(start_ind-1, d), np.arange(0, start_ind-1)), axis=0)
        B = A.transpose(*perm)
        s = B.shape
        unf = B.reshape(s[0], np.prod(s[1:]), order='F')
        U, S, V = np.linalg.svd(unf)
        rnew = truncation_index(S,norm_error*np.linalg.norm(S))-1
        factors = divisors(rnew)
        l = len(factors)
        
        for i in range(l):
            r0 = factors[i]
            trtmp = TRdecomp(B, prec, r0)
            
            a = num_param_tr(trtmp)
            if a < curr_small:
                tr = trtmp
                curr_ind = start_ind
                curr_small = a
           
    # unpermute the cores
    res = ipermuteTR(tr, curr_ind)
    
    return res

### TR heuristic

In [9]:
def interaction_rank(A, k):
    # computes the k:th interaction rank of T
    d = np.ndim(A)
    s = A.shape
    n_ind = k + 1
    if k == d-1:
        n_ind = 0
    perm = np.concatenate((np.arange(k, d), np.arange(0, k)), axis=0)
    T_r = A.transpose(*perm)
    T_r = T_r.reshape(int(s[k]*s[n_ind]), int(np.size(A)/(s[k]*s[n_ind])), order='F')
    res = np.linalg.matrix_rank(T_r)
    return res


def choose_starting_index(A): 
    # Chooses the starting index k minimizing rank(IM_k)
    d = np.ndim(A)
    i_ranks = np.zeros(d)
    for k in range(d):
        i_ranks[k] = interaction_rank(A, k)
    res = np.argmin(i_ranks)
    
    return res, i_ranks

In [10]:
def tr_heuristic(A, prec):
    """Converts a tensor A in full format into TR-format.
    Cyclic shift and divisor of first unfolding rank of A
    are chosen heuristically.
    Args:
    A: a tensor in full format 
    prec: a relative accuracy
    Returns
    TR-format tensor representation 
    """
    d = np.ndim(A)
    norm_error = prec/np.sqrt(d-1)
    cores = [[None] * 1  for _ in range(d)]
    final_cores = [[None] * 1  for _ in range(d)]
    
    # pick initial permutation
    [start_ind, i_ranks] = choose_starting_index(A)
    print(start_ind) 
    perm = np.concatenate((np.arange(start_ind, d), np.arange(0, start_ind)), axis=0)
    print(perm)
    A = A.transpose(*perm)
    s = A.shape
    
    C = A.copy()
    rold = 1
    
    # initial step
    n = s[0]
    C = C.reshape(int(rold*n), int(np.size(C)/(rold*n)), order='F');
    U, S, Vh = svd_fix(C)
    V = Vh.T
    rnew = truncation_index(S, norm_error*np.linalg.norm(S))-1

    factors = divisors(rnew)
    n_ind = start_ind - 1
    if n_ind == -1:
        n_ind = d-1
    
    tmp = abs(rnew / factors - i_ranks[n_ind]) + abs(factors-i_ranks[start_ind])
    m_ind = np.argmin(tmp)
    r0 = factors[m_ind]
    
    U = U[:,:rnew]
    S = S[:rnew]
    V = V[:, :rnew]
    
    Unew = np.zeros((int(rold*r0), n, int(rnew/r0)))
    for index in range(1, r0+1):
        Unew[index - 1, :, :] = U[:, int((index-1)*(rnew//r0)):int(index*(rnew//r0))]
    
    cores[0] = Unew.copy()
    C = np.diag(S) @ V.T
    
    # reshape C into a 3-tensor
    Cnew = np.zeros((int(rnew//r0), int(np.size(C)//(rnew)), r0))
    for index in range(1, r0+1):
        Cnew[:, :, index-1] = C[int((index-1)*(rnew//r0) + 1-1):int(index*(rnew/r0)), :]
    
    C = Cnew.copy()
    shapes = s[1:]
    C = C.reshape(int(rnew/r0), *shapes, r0, order='F')
    shapes = s[1:-1]
    C = C.reshape(int(rnew/r0), *shapes, int(s[-1]*r0), order='F')
    C = C.reshape(int(rnew/r0), int(np.size(C)//(rnew//r0)), order='F')
    rold = int(rnew/r0)
    
    for k in range(1, d-1):
        n = s[k]
        C = C.reshape(int(rold*n), int(np.size(C)/(rold*n)),  order='F')     
        U, S, V = svd_fix(C)
        rnew = truncation_index(S, norm_error*np.linalg.norm(S))-1
        U = U[:,0:rnew]
        S = S[0:rnew]
        V = V[:, 0:rnew]
        cores[k] = U.reshape(rold, n, rnew, order='F')
        C = np.diag(S) @ V.T
        rold = rnew    
    C = C.reshape(rold, s[-1], r0, order='F')
    cores[d-1] = C
    
    # unpermute the cores      
    perm = np.concatenate((np.arange(start_ind, d), np.arange(0, start_ind)), axis=0)
    inv_perm = np.arange(len(perm))
    
    for i in range(len(perm)): 
        inv_perm[perm[i]] = i
        
    for k in range(d):
        final_cores[k] = cores[inv_perm[k]]
    
    return final_cores

### Experimrnts

In [43]:
d = 5 
n =20 
eps = 10e-12

In [45]:
m_deg = 4
m_term = 6

In [54]:
def get_alpha(m_term, d, m_deg): 
    alpha = np.zeros((m_term, d))
    for i in range(m_term):
        for j in range(d):
            alpha[i, j] = np.random.uniform(1, m_deg)
    return alpha

In [55]:
alpha = np.zeros((m_term, d))

for i in range(m_term):
    for j in range(d):
        alpha[i, j] = np.random.uniform(1, m_deg)

In [58]:
X = np.zeros((n, n, n, n, n))

In [65]:
def get_X(n, alpha, m_term):
    X = np.zeros((n, n, n, n, n))
    for i0, x0 in enumerate(np.linspace(0, 1, n)):
        for i1, x1 in enumerate(np.linspace(0, 1, n)):
            for i2, x2 in enumerate(np.linspace(0, 1, n)):
                for i3, x3 in enumerate(np.linspace(0, 1, n)):
                    for i4, x4 in enumerate(np.linspace(0, 1, n)):
                        for i in range(m_term):
                            X[i0, i1, i2, i3, i4] += np.power(x0, alpha[i, 0]) * np.power(x1, alpha[i, 1]) * np.power(x2, alpha[i, 2]) * np.power(x3, alpha[i, 3]) * np.power(x4, alpha[i, 4])

In [66]:
m_deg = np.array([2, 4, 6, 8, 10])
m_term = np.array([2, 6, 10, 14, 18])

In [73]:
for i in range(len(m_deg)):
    for j in range(len(m_term)): 
        tr_costs = []
        tr_shift_cots = []
        tr_smart_costs = []
        for l in range(100):
            alpha = get_alpha(m_term[j], d, m_deg[i])
            X_tensor = get_X(n, alpha, m_term[j])
            tt_svd = TRdecomp(X_tensor, eps, 1)

            _,_,r1 = tt_svd[0].shape
            divs = divisors(r1)
            tmp = list(abs(divs - r1/divs)) #balanced choice
            r0 = tmp.index(min(tmp)) 

            tr_svd = TRdecomp(X_tensor, eps, divs[r0])
            tr_svd_shifts = tr_shifts(X_tensor, eps)
            tr_heurist = tr_heuristic(X_tensor, eps)

            tr_svd_cost = num_param_tr(tr_svd)/num_param_tr(tt_svd)
            tr_shift_cost = num_param_tr(tr_svd_shifts)/num_param_tr(tt_svd)
            tr_heur_cost = num_param_tr(tr_heurist)/num_param_tr(tt_svd)
            
            tr_costs.append(tr_svd_cost)
            tr_shift_cots.append(tr_shift_cost)
            tr_smart_costs.append(tr_heur_cost)
            
        print('m_deg {}, m_term {}, TR-SVD {:.2f}, TT_shifts {:.2f}, TR_heurist{:.2f}'.format(m_deg[i], m_term[j], np.array(tr_costs).mean(), np.array(tr_shift_cots).mean(), np.array(tr_smart_costs).mean()))

m_deg 2, m_term 2, TR-SVD 1, TT_shifts 0.90, TR_heurist 0.94
m_deg 2, m_term 6, TR-SVD 1, TT_shifts 0.91, TR_heurist 0.97
m_deg 2, m_term 10, TR-SVD 1, TT_shifts 0.93, TR_heurist 0.95
m_deg 2, m_term 14, TR-SVD 1, TT_shifts 0.97, TR_heurist 0.97
m_deg 2, m_term 18, TR-SVD 1, TT_shifts 0.98, TR_heurist 0.98
m_deg 4, m_term 2, TR-SVD 1, TT_shifts 0.92, TR_heurist 0.90
m_deg 4, m_term 6, TR-SVD 1.48, TT_shifts 0.86, TR_heurist 0.95
m_deg 4, m_term 10, TR-SVD 1.68, TT_shifts 0.88, TR_heurist 0.93
m_deg 4, m_term 14, TR-SVD 1.73, TT_shifts 0.92, TR_heurist 0.95
m_deg 4, m_term 18, TR-SVD 1.65, TT_shifts 0.91, TR_heurist 0.93
m_deg 6, m_term 2, TR-SVD 1, TT_shifts 0.95, TR_heurist 0.98
m_deg 6, m_term 6, TR-SVD 1.62, TT_shifts 0.88, TR_heurist 0.93
m_deg 6, m_term 10, TR-SVD 1.63, TT_shifts 0.89, TR_heurist 0.95
m_deg 6, m_term 14, TR-SVD 1.89, TT_shifts 0.91, TR_heurist 0.94
m_deg 6, m_term 18, TR-SVD 1.96, TT_shifts 0.91, TR_heurist 0.95
m_deg 8, m_term 2, TR-SVD 1, TT_shifts 0.94, TR_heur

In [75]:
for i in range(len(m_deg)):
    for j in range(len(m_term)): 
        tr_costs = []
        tr_shift_cots = []
        tr_smart_costs = []
        for l in range(50):
            alpha = get_alpha(m_term[j], d, m_deg[i])
            X_tensor = get_X(n, alpha, m_term[j])
            tt_svd = TRdecomp(X_tensor, eps, 1)

            _,_,r1 = tt_svd[0].shape
            divs = divisors(r1)
            tmp = list(abs(divs - r1/divs)) #balanced choice
            r0 = tmp.index(min(tmp)) 

            tr_svd = TRdecomp(X_tensor, eps, divs[r0])
            divs = divisors(r1)
            tr_svd_min = TRdecomp(X_tensor, eps, divs[0])
            for r in divs[1:]:
                tr_svd_ex = Rdecomp(X_tensor, eps, r)
                if num_param_tr(tr_svd_ex) < num_param(tr_svd_min):
                    tr_svd_min = tr_svd_ex
            
            tr_heurist = tr_heuristic(X_tensor, eps)
            
            tr_svd_cost = num_param_tr(tr_svd)/num_param_tr(tt_svd)
            tr_shift_cost = num_param_tr(tr_svd_min)/num_param_tr(tt_svd)
            tr_heur_cost = num_param_tr(tr_heurist)/num_param_tr(tt_svd)
            
            tr_costs.append(tr_svd_cost)
            tr_shift_cots.append(tr_shift_cost)
            tr_smart_costs.append(tr_heur_cost)
            
        print('m_deg {}, m_term {}, TR-SVD {:.2f}, TR_search {:.2f}, TR_heurist{:.2f}'.format(m_deg[i], m_term[j], np.array(tr_costs).mean(), np.array(tr_shift_cots).mean(), np.array(tr_smart_costs).mean()))

m_deg 2, m_term 2, TR-SVD 1, TR_search 0.94, TR_heurist 0.95
m_deg 2, m_term 6, TR-SVD 1, TR_search  0.94, TR_heurist 0.98
m_deg 2, m_term 10, TR-SVD 1, TR_search 0.95, TR_heurist 0.95
m_deg 2, m_term 14, TR-SVD 1, TR_search 0.98, TR_heurist 0.98
m_deg 2, m_term 18, TR-SVD 1, TR_search 0.98, TR_heurist 0.99
m_deg 4, m_term 2, TR-SVD 1, TR_search 0.95, TR_heurist 0.99
m_deg 4, m_term 6, TR-SVD 1.46, TR_search 0.93, TR_heurist 0.96
m_deg 4, m_term 10, TR-SVD 1.67, TR_search 0.93, TR_heurist 0.95
m_deg 4, m_term 14, TR-SVD 1.74, TR_search 0.94, TR_heurist 0.96
m_deg 4, m_term 18, TR-SVD 1.64, TR_search 0.94, TR_heurist 0.94
m_deg 6, m_term 2, TR-SVD 1, TR_search 0.97, TR_heurist 0.99
m_deg 6, m_term 6, TR-SVD 1.62, TR_search 0.93, TR_heurist 0.93
m_deg 6, m_term 10, TR-SVD 1.63, TR_search 0.94, TR_heurist 0.96
m_deg 6, m_term 14, TR-SVD 1.90, TR_search 0.95, TR_heurist 0.95
m_deg 6, m_term 18, TR-SVD 1.94, TR_search 0.94, TR_heurist 0.96
m_deg 8, m_term 2, TR-SVD 1, TR_search 0.95, TR_heu

In [11]:
d = 5
a = 0
b = 1
n = 10
N = n**d
h = (b-a)/n
e = np.ones((1,n))
x1 = np.linspace(a, b, n)
x = np.zeros((d,N))

In [12]:
x[0,:] = np.kron(np.kron(np.kron(np.kron(e,e),e),e),x1)
x[1,:] = np.kron(np.kron(np.kron(np.kron(e,e),e),x1),e)
x[2,:] = np.kron(np.kron(np.kron(np.kron(e,e),x1),e),e)
x[3,:] = np.kron(np.kron(np.kron(np.kron(e,x1),e),e),e)
x[4,:] = np.kron(np.kron(np.kron(np.kron(x1,e),e),e),e)

In [13]:
# A = np.exp(x[0,:]*x[1,:] + x[1,:]*x[2,:] + x[2,:]*x[3,:] + x[1,:]*x[4,:])
# A = 1./np.sqrt(1 + x[0,:]**2+ x[1,:]**2+ x[2,:]**2 + x[3,:]**2 + x[4,:]**2)
# A = np.exp(np.cos(x[1,:]*x[5,:] + x[2,:]*x[1,:] + x[4,:]+x[3,:]));
# A = np.exp(((x[1, :]*x[2,:] + x[5,:]*x[4,:]*x[3,:]*x[1,:]**10)));
A = np.exp(np.cos(x[0,:]*x[4,:] + x[1,:]+x[2,:]+ x[3,:]))

In [14]:
A = A.reshape(10, 10, 10, 10, 10 , order='F')

In [15]:
prec = 1e-12
max_iter = 1

In [16]:
import time

In [17]:
# TT decomp
# start_time = time.time()
for k in range(max_iter):
    zTT = TRdecomp(A, prec, 1)
# t1=time.time() - start_time
# print(t1)

In [19]:
_,_,r1 = zTT[0].shape
divs = divisors(r1)
tmp = list(abs(divs - r1/divs))
r0 = tmp.index(min(tmp)) 

In [21]:
# TR SVD
start_time = time.time()
for k in range(max_iter):
    zBal = TRdecomp(A, prec, divs[r0])
tbal=time.time() - start_time
print(tbal)

10.271489143371582


In [22]:
# TR with shifts
start_time = time.time()
for k in range(max_iter):
    zTR = tr_shifts(A, prec)
tall = time.time() - start_time
print(tall)

88.81568503379822


In [23]:
# TR with shifts heuristic
start_time = time.time()
for k in range(max_iter):
    zPrealt2 = tr_heuristic(A, prec)
tprealt2=time.time() - start_time
print(tprealt2)

4
[4 0 1 2 3]
4.206234931945801


In [379]:
# TR with shifts heuristic
start_time = time.time()
for k in range(max_iter):
    zPrealt2 = tr_heuristic(A, prec)
tprealt2=time.time() - start_time
print(tprealt2)

4
[4 0 1 2 3]
4.329229831695557


In [25]:
print('Storage quotients')
print('TR SVD vs TT {:.2f}'.format(num_param_tr(zBal)/num_param_tr(zTT)))
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))
print('TR heuristec vs TT {:.2f}'.format(num_param_tr(zPrealt2)/num_param_tr(zTT)))

Storage quotients
TR SVD vs TT 0.5311407928139035
TR with shifts vs TT 0.06902636203866433
TR heuristec vs TT 0.06902636203866433


In [30]:
print('Storage quotients')
print('TR SVD vs TT {:.2f}'.format(num_param_tr(zBal)/num_param_tr(zTT)))
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))
print('TR heuristec vs TT {:.2f}'.format(num_param_tr(zPrealt2)/num_param_tr(zTT)))

Storage quotients
TR SVD vs TT 1.29
TR with shifts vs TT 0.74
TR heuristec vs TT 0.90


In [32]:
print('Storage quotients')
print('TR SVD vs TT {:.2f}'.format(num_param_tr(zBal)/num_param_tr(zTT)))
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))
print('TR heuristec vs TT {:.2f}'.format(num_param_tr(zPrealt2)/num_param_tr(zTT)))

Storage quotients
TR SVD vs TT 1.93
TR with shifts vs TT 0.44
TR heuristec vs TT 0.53


In [41]:
print('Storage quotients')
print('TR SVD vs TT {:.2f}'.format(num_param_tr(zBal)/num_param_tr(zTT)))
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))
print('TR heuristec vs TT {:.2f}'.format(num_param_tr(zPrealt2)/num_param_tr(zTT)))

Storage quotients
TR SVD vs TT 1.29
TR with shifts vs TT 0.74
TR heuristec vs TT 0.90


In [42]:
print('Storage quotients')
print('TR SVD vs TT {:.2f}'.format(num_param_tr(zBal)/num_param_tr(zTT)))
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))
print('TR heuristec vs TT {:.2f}'.format(num_param_tr(zPrealt2)/num_param_tr(zTT)))

Storage quotients
TR SVD vs TT 1.90
TR with shifts vs TT 0.28
TR heuristec vs TT 0.28


In [40]:
# experiments with diffent cyclic shifts
_,_,r1 = zTT[0].shape
divs = divisors(r1)
# tmp = list(abs(divs - r1/divs))
# r0 = tmp.index(min(tmp))

In [76]:
# TT svd
zTT = TRdecomp(A, prec, 1)
zTR = TRdecomp(A, prec, divs[-1])

In [34]:
print('Storage quotients k=1')
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))

Storage quotients k=1
TR with shifts vs TT 1.00


In [35]:
print('Storage quotients k=2')
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))

Storage quotients k=2
TR with shifts vs TT 0.44


In [36]:
print('Storage quotients k=3')
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))

Storage quotients k=3
TR with shifts vs TT 0.72


In [38]:
print('Storage quotients k=4')
print('TR with shifts vs TT {:.2f}'.format(num_param_tr(zTR)/num_param_tr(zTT)))

Storage quotients k=4
TR with shifts vs TT 0.91
